# 1 ASOS_DATA PROCESSING

In [2]:
# Basic setting for Jupyter_notebook to import utils
import os
import sys

notebook_path = os.path.abspath("")
project_root = os.path.abspath(os.path.join(notebook_path, "../../"))

sys.path.append(project_root)

In [46]:
import os
import sys
import pandas as pd
from utils import folder_utils
from tqdm import tqdm

In [47]:
def extract_data_to_df(country, data_folder, data_category, output_folder):
    # Specify the folder path
    input_folder_path = folder_utils.find_folder(
        country, data_folder, data_category, output_folder
    )
    # Initialize a dataframe to store all the data from the same country
    raw_df = pd.DataFrame()
    # Get the filenames of all CSV files under the folder except the station network file
    csv_files = [
        f
        for f in os.listdir(input_folder_path)
        if f.endswith('.csv') and "asos_station_network" not in f
    ]
    # Read and merge the csv files in queue
    for csv_file in tqdm(csv_files):
        csv_file_path = os.path.join(input_folder_path, csv_file)
        df = pd.read_csv(csv_file_path)
        raw_df = pd.concat([raw_df, df], ignore_index=True)

    return raw_df

In [48]:
def process_asos_rawdata(df):
    # Split "valid" column into "date" and "time" columns
    df['date'] = (
        df['valid'].str.split(' ', expand=True)[0].str.replace('-', '').astype(int)
    )
    df['time'] = df['valid'].str.split(' ', expand=True)[1]

    # Convert Fahrenheit to Kelvin for "tmpf", "feel" and "dwpf" columns
    df['tmpf'] = (df['tmpf'] - 32) * 5 / 9 + 273.15
    df['tmpf'] = df['tmpf'].round(1)

    df['dwpf'] = (df['dwpf'] - 32) * 5 / 9 + 273.15
    df['dwpf'] = df['dwpf'].round(1)

    # Convert Fahrenheit to Kelvin for "feel" column
    df['feel'] = (df['feel'] - 32) * 5 / 9 + 273.15
    df['feel'] = df['feel'].round(1)
    
    # Convert knots to m/s for "sknt" and "gust" columns
    df['sknt'] = df['sknt'] * 0.514444
    df['gust'] = df['gust'] * 0.514444

    # Convert inches to meters for "p01i" and "alti" columns
    df['p01i'] = pd.to_numeric(df['p01i'], errors='coerce') * 0.0254
    df['alti'] = df['alti'] * 0.0254

    # Convert millibar to Pa for "mslp" column
    df['mslp'] = df['mslp'] * 100

    # Drop columns
    columns_to_drop = [
        # 'lon',
        # 'lat',
        # 'elevation',
        'valid',
        'skyc1',
        'skyc2',
        'skyc3',
        'skyc4',
        'skyl1',
        'skyl2',
        'skyl3',
        'skyl4',
        'wxcodes',
        'ice_accretion_1hr',
        'ice_accretion_3hr',
        'ice_accretion_6hr',
        'peak_wind_gust',
        'peak_wind_drct',
        'peak_wind_time',
        'metar',
        'snowdepth',
    ]
    df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

    return df

In [49]:
def save_asos_processed_data(
    processed_df, country, data_folder, data_category, output_folder
):
    output_directory = folder_utils.create_folder(
        country, data_folder, data_category, output_folder
    )
    output_filename = f"{country}_ASOS_processed_data.csv"
    output_filepath = os.path.join(output_directory, output_filename)
    processed_df.to_csv(output_filepath, index=False, encoding="utf-8")
    print(f'{output_filename} done!')

In [50]:
# Example usage

country = "GB"
data_folder = "data"
data_test_category = "test_data"
data_read_category = "raw_data"
data_save_category = "processed_data"
output_folder = "ASOS_DATA"

In [51]:
raw_df = extract_data_to_df(country, data_folder, data_read_category , output_folder)

  3%|█▎                                               | 3/112 [00:00<00:10, 10.84it/s]/var/folders/tt/fs_l1by51m30s907pzxq6gf40000gp/T/ipykernel_38549/240348863.py:17: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file_path)
  4%|██▏                                              | 5/112 [00:00<00:11,  9.60it/s]/var/folders/tt/fs_l1by51m30s907pzxq6gf40000gp/T/ipykernel_38549/240348863.py:17: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file_path)
  6%|███                                              | 7/112 [00:00<00:08, 12.02it/s]/var/folders/tt/fs_l1by51m30s907pzxq6gf40000gp/T/ipykernel_38549/240348863.py:17: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file_path)
 10%|████▋                                           | 11/112 [00:00<00:08, 11.66it/s]/var/

In [52]:
raw_df

,station,valid,lon,lat,elevation,tmpf,dwpf,relh,drct,sknt,...,wxcodes,ice_accretion_1hr,ice_accretion_3hr,ice_accretion_6hr,peak_wind_gust,peak_wind_drct,peak_wind_time,feel,metar,snowdepth
0,EGUN,2022-01-01 00:26,0.4864,52.3619,10.0,53.6,53.6,100.00,210.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.6,EGUN 010026Z AUTO 21009KT 9999 BKN013 12/12 A3...,NaN
1,EGUN,2022-01-01 00:56,0.4864,52.3619,10.0,54.0,54.0,100.00,210.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.0,EGUN 010056Z AUTO 21010KT 9999 SCT012 BKN035 1...,NaN
2,EGUN,2022-01-01 01:26,0.4864,52.3619,10.0,55.4,55.4,100.00,200.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.4,EGUN 010126Z AUTO 20009KT 9999 SCT034 13/13 A3...,NaN
3,EGUN,2022-01-01 01:56,0.4864,52.3619,10.0,54.7,54.7,100.00,210.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.7,EGUN 010156Z AUTO 21011KT 9999 CLR 13/13 A3008...,NaN
4,EGUN,2022-01-01 02:56,0.4864,52.3619,10.0,55.6,55.6,100.00,200.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.6,EGUN 010256Z AUTO 20009KT 9999 CLR 13/13 A3007...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2038806,EGKB,2023-08-01 19:20,0.0325,51.3308,183.0,60.8,57.2,87.92,210.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.8,EGKB 011920Z 21011KT 170V240 9999 FEW015 16/14...,NaN
2038807,EGKB,2023-08-01 19:50,0.0325,51.3308,183.0,60.8,55.4,82.37,210.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.8,EGKB 011950Z 21010KT 9999 FEW015 16/13 Q1001,NaN
2038808,EGKB,2023-08-01 20:20,0.0325,51.3308,183.0,59.0,55.4,87.83,200.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.0,EGKB 012020Z 20011KT 9999 FEW025 15/13 Q1001,NaN
2038809,EGKB,2023-08-01 20:50,0.0325,51.3308,183.0,59.0,55.4,87.83,210.0,11.0,...,RA,NaN,NaN,NaN,NaN,NaN,NaN,59.0,EGKB 012050Z 21011KT 9999 RA SCT050 15/13 Q1001,NaN


In [53]:
processed_df = process_asos_rawdata(raw_df)

In [54]:
processed_df

,station,lon,lat,elevation,tmpf,dwpf,relh,drct,sknt,p01i,alti,mslp,vsby,gust,feel,date,time
0,EGUN,0.4864,52.3619,10.0,285.2,285.2,100.00,210.0,4.629996,0.0,0.764032,101890.0,6.21,NaN,285.2,20220101,00:26
1,EGUN,0.4864,52.3619,10.0,285.4,285.4,100.00,210.0,5.144440,0.0,0.764032,101890.0,6.21,NaN,285.4,20220101,00:56
2,EGUN,0.4864,52.3619,10.0,286.2,286.2,100.00,200.0,4.629996,0.0,0.763778,101860.0,6.21,NaN,286.2,20220101,01:26
3,EGUN,0.4864,52.3619,10.0,285.8,285.8,100.00,210.0,5.658884,0.0,0.764032,101890.0,6.21,NaN,285.8,20220101,01:56
4,EGUN,0.4864,52.3619,10.0,286.3,286.3,100.00,200.0,4.629996,0.0,0.763778,101860.0,6.21,NaN,286.3,20220101,02:56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2038806,EGKB,0.0325,51.3308,183.0,289.2,287.2,87.92,210.0,5.658884,0.0,0.750824,NaN,6.21,NaN,289.2,20230801,19:20
2038807,EGKB,0.0325,51.3308,183.0,289.2,286.2,82.37,210.0,5.144440,0.0,0.750824,NaN,6.21,NaN,289.2,20230801,19:50
2038808,EGKB,0.0325,51.3308,183.0,288.2,286.2,87.83,200.0,5.658884,0.0,0.750824,NaN,6.21,NaN,288.2,20230801,20:20
2038809,EGKB,0.0325,51.3308,183.0,288.2,286.2,87.83,210.0,5.658884,0.0,0.750824,NaN,6.21,NaN,288.2,20230801,20:50


In [27]:
save_asos_processed_data(
    processed_df, country, data_folder, data_save_category, output_folder
)

Folder '/Users/ww721/JupyterNotebookPath/IRP_20220602/irp_ww721_bakcup/data/processed_data/ASOS_DATA/GB_ASOS_DATA' created successfully.
GB_ASOS_processed_data.csv done!


In [ ]:
/Users/ww721/JupyterNotebookPath/IRP_20220602/irp_ww721_bakcup/data/test_data/ASOS_DATA
/Users/ww721/JupyterNotebookPath/IRP_20220602/irp_ww721_bakcup/data/test_data/ASOS_DATA/GB_ASOS_DATA'

---

---

# 2 ERA5_DATA_PROCESSING

In [3]:
import os
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from utils import folder_utils
from tqdm import tqdm

In [4]:
# Example usage

country = "GB"
data_folder = "data"
data_test_category = "test_data"
data_read_category = "raw_data"
data_save_category = "processed_data"
# output_folder = "ASOS_DATA"
output_folder2 ="ERA5_DATA"

In [5]:
folder_utils.create_folder( country, data_folder, data_test_category, output_folder2)

Folder '/Users/ww721/JupyterNotebookPath/IRP_20220602/irp_ww721_bakcup/data/test_data/ERA5_DATA/GB_ERA5_DATA' created successfully.


'/Users/ww721/JupyterNotebookPath/IRP_20220602/irp_ww721_bakcup/data/test_data/ERA5_DATA/GB_ERA5_DATA'

In [11]:
def extract_data_nc_to_xr(country, data_folder, data_category, output_folder):
    # Specify the folder path
    input_folder_path = folder_utils.find_folder(
        country, data_folder, data_category, output_folder
    )
    raw_ds = None  # Initialize raw_ds outside the loop
    nc_files = [
        f
        for f in os.listdir(input_folder_path)
        if f.endswith('.nc') and "era5_station_network" not in f
    ]
    # Read and merge the csv files in queue
    for nc_file in tqdm(nc_files):
        nc_file_path = os.path.join(input_folder_path, nc_file)
        raw_ds = xr.open_dataset(nc_file_path)
        # raw_df = pd.concat([raw_df, df], ignore_index=True)

    return raw_ds


In [13]:
ds = extract_data_nc_to_xr(country, data_folder, data_test_category, output_folder2)

100%|█████████████████████████████████████████████████| 1/1 [00:00<00:00, 17.73it/s]


In [14]:
print(ds)

<xarray.Dataset>
Dimensions:    (longitude: 41, latitude: 45, time: 744)
Coordinates:
  * longitude  (longitude) float32 -8.0 -7.75 -7.5 -7.25 ... 1.25 1.5 1.75 2.0
  * latitude   (latitude) float32 61.0 60.75 60.5 60.25 ... 50.5 50.25 50.0
  * time       (time) datetime64[ns] 2023-01-01 ... 2023-01-31T23:00:00
Data variables:
    u10        (time, latitude, longitude) float32 ...
    v10        (time, latitude, longitude) float32 ...
    d2m        (time, latitude, longitude) float32 ...
    t2m        (time, latitude, longitude) float32 ...
    i10fg      (time, latitude, longitude) float32 ...
    msl        (time, latitude, longitude) float32 ...
    skt        (time, latitude, longitude) float32 ...
    sp         (time, latitude, longitude) float32 ...
    tp         (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-08-14 13:08:35 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...


In [16]:
ds.dims

Frozen({'longitude': 41, 'latitude': 45, 'time': 744})

In [17]:
ds.data_vars

Data variables:
    u10      (time, latitude, longitude) float32 ...
    v10      (time, latitude, longitude) float32 ...
    d2m      (time, latitude, longitude) float32 ...
    t2m      (time, latitude, longitude) float32 ...
    i10fg    (time, latitude, longitude) float32 ...
    msl      (time, latitude, longitude) float32 ...
    skt      (time, latitude, longitude) float32 ...
    sp       (time, latitude, longitude) float32 ...
    tp       (time, latitude, longitude) float32 ...

In [18]:
ds.coords

Coordinates:
  * longitude  (longitude) float32 -8.0 -7.75 -7.5 -7.25 ... 1.25 1.5 1.75 2.0
  * latitude   (latitude) float32 61.0 60.75 60.5 60.25 ... 50.5 50.25 50.0
  * time       (time) datetime64[ns] 2023-01-01 ... 2023-01-31T23:00:00

In [19]:
ds.attrs

{'Conventions': 'CF-1.6',
 'history': '2023-08-14 13:08:35 GMT by grib_to_netcdf-2.25.1: /opt/ecmwf/mars-client/bin/grib_to_netcdf.bin -S param -o /cache/data9/adaptor.mars.internal-1692018500.2390654-25204-1-38ad28e7-7189-4dcf-abc4-d4ef8d4cf8e3.nc /cache/tmp/38ad28e7-7189-4dcf-abc4-d4ef8d4cf8e3-adaptor.mars.internal-1692018168.885484-25204-1-tmp.grib'}

In [20]:
ds.longitude

<xarray.DataArray 'longitude' (longitude: 41)>
array([-8.  , -7.75, -7.5 , -7.25, -7.  , -6.75, -6.5 , -6.25, -6.  , -5.75,
       -5.5 , -5.25, -5.  , -4.75, -4.5 , -4.25, -4.  , -3.75, -3.5 , -3.25,
       -3.  , -2.75, -2.5 , -2.25, -2.  , -1.75, -1.5 , -1.25, -1.  , -0.75,
       -0.5 , -0.25,  0.  ,  0.25,  0.5 ,  0.75,  1.  ,  1.25,  1.5 ,  1.75,
        2.  ], dtype=float32)
Coordinates:
  * longitude  (longitude) float32 -8.0 -7.75 -7.5 -7.25 ... 1.25 1.5 1.75 2.0
Attributes:
    units:      degrees_east
    long_name:  longitude

In [21]:
ds.latitude

<xarray.DataArray 'latitude' (latitude: 45)>
array([61.  , 60.75, 60.5 , 60.25, 60.  , 59.75, 59.5 , 59.25, 59.  , 58.75,
       58.5 , 58.25, 58.  , 57.75, 57.5 , 57.25, 57.  , 56.75, 56.5 , 56.25,
       56.  , 55.75, 55.5 , 55.25, 55.  , 54.75, 54.5 , 54.25, 54.  , 53.75,
       53.5 , 53.25, 53.  , 52.75, 52.5 , 52.25, 52.  , 51.75, 51.5 , 51.25,
       51.  , 50.75, 50.5 , 50.25, 50.  ], dtype=float32)
Coordinates:
  * latitude  (latitude) float32 61.0 60.75 60.5 60.25 ... 50.75 50.5 50.25 50.0
Attributes:
    units:      degrees_north
    long_name:  latitude

In [22]:
ds.time

<xarray.DataArray 'time' (time: 744)>
array(['2023-01-01T00:00:00.000000000', '2023-01-01T01:00:00.000000000',
       '2023-01-01T02:00:00.000000000', ..., '2023-01-31T21:00:00.000000000',
       '2023-01-31T22:00:00.000000000', '2023-01-31T23:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2023-01-01 ... 2023-01-31T23:00:00
Attributes:
    long_name:  time

In [27]:
ds.d2m.attrs

{'units': 'K', 'long_name': '2 metre dewpoint temperature'}

In [28]:
ds.d2m.long_name

'2 metre dewpoint temperature'

In [29]:
ds.longitude.shape

(41,)

In [34]:
ds.latitude.shape[0]

45

In [37]:
ds.latitude.shape

(45,)

In [32]:
ds.latitude.values[0]

61.0

In [31]:
ds.d2m.shape

(744, 45, 41)

In [39]:
print('Latitudes and longitudes of domain corners:')
pt1 = 0
pt2 = 0
print('  0, 0:        ', ds.latitude.values[pt1], ds.longitude.values[pt2])
pt1 = ds.latitude.shape[0] - 1
print('  0, x-max:    ', ds.latitude.values[pt1], ds.longitude.values[pt2])
pt = ds.latitude.shape[0] - 1
print('  y-max, 0:    ', ds.latitude.values[pt], ds.longitude.values[pt])
pt = ds.latitude.shape[0] - 1
pt2 = ds.longitude.shape[0] - 1
print('  y-max, x-max:', ds.latitude.values[pt], ds.longitude.values[pt2])

Latitudes and longitudes of domain corners:
  0, 0:         61.0 -8.0
  0, x-max:     50.0 -8.0


IndexError: index 44 is out of bounds for axis 0 with size 41

In [42]:
ds.latitude.values # 提取数值

array([61.  , 60.75, 60.5 , 60.25, 60.  , 59.75, 59.5 , 59.25, 59.  ,
       58.75, 58.5 , 58.25, 58.  , 57.75, 57.5 , 57.25, 57.  , 56.75,
       56.5 , 56.25, 56.  , 55.75, 55.5 , 55.25, 55.  , 54.75, 54.5 ,
       54.25, 54.  , 53.75, 53.5 , 53.25, 53.  , 52.75, 52.5 , 52.25,
       52.  , 51.75, 51.5 , 51.25, 51.  , 50.75, 50.5 , 50.25, 50.  ],
      dtype=float32)

In [43]:
ds.latitude.values[3:5]   # like numpy

array([60.25, 60.  ], dtype=float32)

In [62]:
# Data Description
for i in ds.data_vars:
    print("--------------------------------------------------------------------")
    print(i)
    print("--------------------------------------------------------------------")
    var_data = ds[i]
    print(var_data)

--------------------------------------------------------------------
u10
--------------------------------------------------------------------
<xarray.DataArray 'u10' (time: 744, latitude: 45, longitude: 41)>
array([[[ 5.74171 ,  5.559723, ...,  1.951654,  2.056129],
        [ 5.009717,  4.820315, ...,  0.776825,  0.867818],
        ...,
        [ 9.237216,  9.113869, ...,  6.014691,  6.080746],
        [ 9.562098,  9.437403, ...,  5.999863,  6.034238]],

       [[ 6.154889,  5.882582, ...,  1.274257,  1.317394],
        [ 5.358189,  5.070379, ...,  0.153349,  0.197835],
        ...,
        [10.405306, 10.035938, ...,  6.737248,  6.484488],
        [10.566399, 10.195009, ...,  6.33418 ,  6.156911]],

       ...,

       [[18.09124 , 18.360176, ..., 12.231245, 12.469851],
        [20.895866, 21.120317, ..., 12.777208, 13.021879],
        ...,
        [ 7.771881,  7.767837, ...,  4.308728,  3.749959],
        [ 6.876099,  6.87138 , ...,  4.313446,  4.036421]],

       [[16.051632, 16.261

In [59]:
for i in ds.data_vars:
    print(var_data.values)

[[[ 5.74171     5.559723    5.412111   ...  1.9058204   1.9516544
    2.0561285 ]
  [ 5.0097165   4.820315    4.5884495  ...  0.73436093  0.7768247
    0.86781836]
  [ 4.177293    3.962952    3.7021034  ... -0.24163055 -0.1870346
   -0.12435007]
  ...
  [ 9.254741    9.186665    9.167117   ...  6.9805727   6.049741
    6.3328323 ]
  [ 9.237216    9.113869    9.058599   ...  6.8619437   6.0146914
    6.0807457 ]
  [ 9.562098    9.437403    9.448187   ...  5.9432445   5.9998627
    6.034238  ]]

 [[ 6.154889    5.8825817   5.610949   ...  1.2587543   1.274257
    1.3173945 ]
  [ 5.3581886   5.0703793   4.719885   ...  0.12301803  0.1533494
    0.1978352 ]
  [ 4.4206166   4.1051717   3.7364788  ... -0.7693939  -0.7094059
   -0.67503023]
  ...
  [10.108059    9.888327    9.628826   ...  8.693951    7.320956
    7.198283  ]
  [10.405306   10.035938    9.724538   ...  8.018575    6.7372484
    6.4844885 ]
  [10.566399   10.195009    9.959774   ...  6.4635935   6.3341804
    6.156911  ]]

 [[